In [1]:
from dotenv import load_dotenv
from portia.cli import CLIExecutionHooks
from portia import *
import requests
import xml.etree.ElementTree as ET
from pydantic import BaseModel, Field
from typing import Generic, TypeVar, List, Literal
import os
from notion_client import Client
from my_custom_tools.registry import custom_tool_registry
import shutil

load_dotenv(override=True)

# Fetch the Notion API key and set up client
notion_api_key = os.getenv("NOTION_API_KEY")
notion_parent_id = os.getenv("NOTION_PARENT_ID")

# Initialize the Notion client
notion = Client(auth=notion_api_key)

In [2]:
my_config = Config.from_default()
complete_tool_registry = PortiaToolRegistry(my_config) + custom_tool_registry

portia = Portia(config = my_config,
                tools = complete_tool_registry,
                execution_hooks=CLIExecutionHooks(),)

In [3]:
constraints = []
# topic = input("What topic are you interested in covering today?")
# number_of_papers = int(input("How many papers do you want to download?"))
topic = "Holodiffusion"
video_preference = True
rec_reading_preference = True
quiz_preference = True

# Define the path to your papers folder
papers_folder = "papers"

# Ensure the papers folder exists
if not os.path.exists(papers_folder):
    os.makedirs(papers_folder)
else:
    # If it exists, clear its contents
    for filename in os.listdir(papers_folder):
        file_path = os.path.join(papers_folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)  # remove file or symlink
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)  # remove directory
        except Exception as e:
            print(f'Failed to delete {file_path}. Reason: {e}')


task = (
            lambda : f"""You are a research assistant running these tasks: 
                      - Find and download 1 paper on the topic of {topic} using the ArXivTool. 
                      - Run the PDFReaderTool to extract the full text from the pdfs in the local folder.
                      - Use PSTool to create and populate the Page Summary subpage.
                      - From the text, extract the core mathematical and scientific concepts required 
                        to understand the paper. Focus only on generalizable topics that could be included 
                        in a learning pathway or curriculum—avoid content specific to the study's location, 
                        data, or outcomes. List only the overarching topics, with no explanations or extra text.
                      - Then use the TopicSelectorTool on these topics. 
                      - Then use the Notion Tool to create Notion pages for these topics.

                      - {video_preference * "Use the YouTubeTool to find videos on each topic."}
                      - {rec_reading_preference * "Use the RecReadTool to find resources on each topic."}
                      - {quiz_preference * "Use the QuizTool to create quizzes on each topic."}
                      
                      
                        Take into account these constraints: {constraints}
                      """

        )



# Iterate on the plan with the user until they are happy with it
with execution_context(end_user_id="learning_enthusiast",):
    plan = portia.plan(task())
    print("\nHere are the steps in the generated plan:")
    [print(step.model_dump_json(indent=2)) for step in plan.steps]
    ready_to_proceed = False
    while not ready_to_proceed:
        user_input = input("Are you happy with the plan? (y/n):\n")
        if user_input == "y":
            ready_to_proceed = True
        else:
            user_input = input("Any additional guidance for the planner?:\n")
            constraints.append(user_input)
            plan = portia.plan(task())
            print("\nHere are the updated steps in the plan:")
            [print(step.model_dump_json(indent=2)) for step in plan.steps]

    # Execute the plan
    print("\nThe plan will now be executed. Please wait...")
    run = portia.run_plan(plan)
    
    if run.state != PlanRunState.COMPLETE:
        raise Exception(
            f"Plan run failed with state {run.state}. Check logs for details."
        )

2025-04-13 06:10:50.486 | INFO | portia.portia:plan:197 - Running planning_agent for query - You are a research assistant running these tasks: 
                      - Find and download 1 paper on the topic of Holodiffusion using the ArXivTool. 
                      - Run the PDFReaderTool to extract the full text from the pdfs in the local folder.
                      - Use PSTool to create and populate the Page Summary subpage.
                      - From the text, extract the core mathematical and scientific concepts required 
                        to understand the paper. Focus only on generalizable topics that could be included 
                        in a learning pathway or curriculum—avoid content specific to the study's location, 
                        data, or outcomes. List only the overarching topics, with no explanations or extra text.
                      - Then use the TopicSelectorTool on these topics. 
                      - Then use the Notion Tool to create

--- Logging error in Loguru Handler #2 ---
Record was: {'elapsed': datetime.timedelta(seconds=350, microseconds=803638), 'exception': None, 'extra': {}, 'file': (name='portia.py', path='/opt/anaconda3/envs/RA/lib/python3.11/site-packages/portia/portia.py'), 'function': '_execute_plan_run', 'level': (name='INFO', no=20, icon='ℹ️'), 'line': 588, 'message': 'Step output - [{"topic": "Diffusion Models", "page_id": "1d46ccbb-ecba-816d-b07f-ffa30847e7ff", "content": "[[Introduction]]\\nDiffusion models are a class of probabilistic models used to describe the way particles, information, or phenomena spread over time and space. These models are widely applicable in fields such as physics, biology, and finance, providing insights into processes like heat distribution, population dynamics, and market trends.\\n\\n[[Key Definitions]]\\n- **Diffusion Process**: A stochastic process that describes the random movement of particles in a medium, often modeled by differential equations.\\n- **Brownian 

2025-04-13 06:16:39.028 | INFO | portia.portia:_execute_plan_run:551 - Executing step 7: Find relevant videos for each topic based on the created pages. | {'plan': 'plan-9a8e30db-a7b9-4912-8527-8ce10756caf1', 'plan_run': 'prun-903a1d6a-ffbe-408a-9e81-853bdb094723'}


--- Logging error in Loguru Handler #2 ---
Record was: {'elapsed': datetime.timedelta(seconds=394, microseconds=349615), 'exception': None, 'extra': {}, 'file': (name='tool_wrapper.py', path='/opt/anaconda3/envs/RA/lib/python3.11/site-packages/portia/tool_wrapper.py'), 'function': 'run', 'level': (name='INFO', no=20, icon='ℹ️'), 'line': 115, 'message': 'Invoking YouTube Tool with args: {\'topics\': [{\'topic\': \'Diffusion Models\', \'page_id\': \'1d46ccbb-ecba-816d-b07f-ffa30847e7ff\', \'content\': "[[Introduction]]\\nDiffusion models are a class of probabilistic models used to describe the way particles, information, or phenomena spread over time and space. These models are widely applicable in fields such as physics, biology, and finance, providing insights into processes like heat distribution, population dynamics, and market trends.\\n\\n[[Key Definitions]]\\n- **Diffusion Process**: A stochastic process that describes the random movement of particles in a medium, often modeled by

2025-04-13 06:17:27.851 | INFO | portia.portia:_execute_plan_run:588 - Step output - 
2025-04-13 06:17:29.411 | INFO | portia.portia:_execute_plan_run:551 - Executing step 8: Identify recommended resources for each topic linked to the created pages. | {'plan': 'plan-9a8e30db-a7b9-4912-8527-8ce10756caf1', 'plan_run': 'prun-903a1d6a-ffbe-408a-9e81-853bdb094723'}


--- Logging error in Loguru Handler #2 ---
Record was: {'elapsed': datetime.timedelta(seconds=434, microseconds=498893), 'exception': None, 'extra': {}, 'file': (name='tool_wrapper.py', path='/opt/anaconda3/envs/RA/lib/python3.11/site-packages/portia/tool_wrapper.py'), 'function': 'run', 'level': (name='INFO', no=20, icon='ℹ️'), 'line': 115, 'message': 'Invoking RecRead Tool with args: {\'topics\': [{\'topic\': \'Diffusion Models\', \'page_id\': \'1d46ccbb-ecba-816d-b07f-ffa30847e7ff\', \'content\': "[[Introduction]]\\nDiffusion models are a class of probabilistic models used to describe the way particles, information, or phenomena spread over time and space. These models are widely applicable in fields such as physics, biology, and finance, providing insights into processes like heat distribution, population dynamics, and market trends.\\n\\n[[Key Definitions]]\\n- **Diffusion Process**: A stochastic process that describes the random movement of particles in a medium, often modeled by

Processing topic: Diffusion Models
Processing topic: 3D Generative Modeling
Processing topic: Super-resolution Networks
2025-04-13 06:18:09.759 | INFO | portia.portia:_execute_plan_run:588 - Step output - ✅ Recommended Reading added to Notion pages successfully.
2025-04-13 06:18:11.521 | INFO | portia.portia:_execute_plan_run:551 - Executing step 9: Generate quizzes for each topic based on the created pages. | {'plan': 'plan-9a8e30db-a7b9-4912-8527-8ce10756caf1', 'plan_run': 'prun-903a1d6a-ffbe-408a-9e81-853bdb094723'}


--- Logging error in Loguru Handler #2 ---
Record was: {'elapsed': datetime.timedelta(seconds=476, microseconds=482524), 'exception': None, 'extra': {}, 'file': (name='tool_wrapper.py', path='/opt/anaconda3/envs/RA/lib/python3.11/site-packages/portia/tool_wrapper.py'), 'function': 'run', 'level': (name='INFO', no=20, icon='ℹ️'), 'line': 115, 'message': 'Invoking Quiz Tool with args: {\'topics\': [{\'topic\': \'Diffusion Models\', \'page_id\': \'1d46ccbb-ecba-816d-b07f-ffa30847e7ff\', \'content\': "[[Introduction]]\\nDiffusion models are a class of probabilistic models used to describe the way particles, information, or phenomena spread over time and space. These models are widely applicable in fields such as physics, biology, and finance, providing insights into processes like heat distribution, population dynamics, and market trends.\\n\\n[[Key Definitions]]\\n- **Diffusion Process**: A stochastic process that describes the random movement of particles in a medium, often modeled by di

2025-04-13 06:19:14.577 | INFO | portia.portia:_execute_plan_run:588 - Step output - Quizzes created successfully!
2025-04-13 06:19:20.106 | INFO | portia.portia:_log_final_output:618 - Final output: The research assistant executed a series of tasks to explore the topic of Holodiffusion, focusing on the paper "HoloFusion: Towards Photo-realistic 3D Generative Modeling." The tasks included downloading the paper, extracting its full text, and identifying core mathematical and scientific concepts such as Diffusion Models, 3D Generative Modeling, and Super-resolution Networks. These topics were selected for further exploration, with Notion pages created and populated with content. Recommended readings and quizzes were also generated, although no videos were found. The process emphasized the integration of super-resolution networks and view consistency in 3D modeling.
